In [1]:
import numpy
import pandas
import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.model_selection
import xgboost

print("Done")

Done


In [2]:
# General setup.

# Return mean absolute error scale.
def mean_absolute_error_salary_scale(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        numpy.exp(y_test), numpy.exp(y_predicted))

def mae_without_log_scaling(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        y_test, y_predicted)

In [3]:
# Define training and test data.

data = pandas.read_csv('data/train.csv')

X_train_index, X_test_index, Y_train, Y_test = (
    sklearn.model_selection.train_test_split(
        data.index, data['LogSalaryNormalized'],
        test_size=.3, random_state=42))

# Define BOW data.
TRAINING_COLUMN = 'CompleteJobListing'

X_train_bow = data.iloc[X_train_index]
X_test_bow = data.iloc[X_test_index]

count_vect = sklearn.feature_extraction.text.CountVectorizer(
    stop_words='english', min_df=50, decode_error='ignore')

# Transform BOW test set.
X_train_counts = count_vect.fit_transform(
    X_train_bow[TRAINING_COLUMN])
X_test_counts = count_vect.transform(
    X_test_bow[TRAINING_COLUMN])

print(X_test_counts.shape)

(51445, 10332)


In [4]:
# Define LDA data.
lda_raw = pandas.read_csv('data/train.csv')

lda_data = pandas.read_csv('data/train_large_lda_50.csv')

X_train_lda_index, X_test_lda_index, Y_train_lda, Y_test_lda = (
    sklearn.model_selection.train_test_split(
        lda_raw.index, lda_raw['LogSalaryNormalized'],
        test_size=.3, random_state=42))

X_train_lda = lda_data.iloc[X_train_index]
X_test_lda = lda_data.iloc[X_test_index]

IndexError: positional indexers are out-of-bounds

In [13]:
# Model: Random Forests
# Data Representation: LDA
# Library used: sklearn

# NOTE: Have since retrained LDA using sklearn and online mini-batch learning.

rf = sklearn.ensemble.RandomForestRegressor(
    n_estimators=1000, n_jobs=-1, warm_start=True)

rf.fit(X_train_lda, Y_train)
rf_predictions = rf.predict(X_test_lda)

# Scale to get Mean Absolute Error
rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)

print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))

Random Forest Regressor Mean Absolute Error:  8380.5428


### Interesting note: Using LDA, n_estimators = 10 performs better than 1000

In [4]:
# Model: XGBoost
# Data Representation: BOW
# Library used: xgboost

# NOTE: xgboost will not import on Rob's laptop.

xgb = xgboost.XGBRegressor(
       n_estimators=1000, learning_rate=.10, max_depth=10, silent=False,
       subsample=.8, colsample_bytree=.8, min_child_weight=.5)

#xgb = xgboost.XGBRegressor(n_estimators=100)

xgb.fit(X_train_counts, Y_train)
xgb_predictions = xgb.predict(X_test_counts)

# Scale to get Mean Absolute Error
xgb_mae = mean_absolute_error_salary_scale(Y_test, xgb_predictions)

print('XGBoost Mean Absolute Error: {:10.4f}'.format(xgb_mae))

XGBoost Mean Absolute Error:  5932.1473


In [19]:
# Model: XGBoost
# Data Representation: LDA
# Library used: xgboost

# NOTE: xgboost will not import on Rob's laptop.

xgb = xgboost.XGBRegressor(
       n_estimators=1000, learning_rate=.10, max_depth=10, silent=False,
       subsample=.8, colsample_bytree=.8, min_child_weight=.5)

xgb.fit(X_train_lda, Y_train_lda)
xgb_predictions = xgb.predict(X_test_lda)

# Scale to get Mean Absolute Error
xgb_mae = mean_absolute_error_salary_scale(Y_test_lda, xgb_predictions)

print('XGBoost Mean Absolute Error: {:10.4f}'.format(xgb_mae))

XGBoost Mean Absolute Error:  8295.2584


In [26]:
# Model: SGD
# Data Representation: LDA
# Library used: sklearn

# Normalization not required as LDA output is normalised between 0 and
# 1 anyway.

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(
    alpha=.0001, penalty='l1', n_iter=10000)
sgd.fit(X_train_lda, Y_train)
sgd_predictions = sgd.predict(X_test_lda)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))

SGDRegressor Mean Absolute Error:  8929.7799


In [5]:
# Model: SGD
# Data Representation: BOW
# Library used: sklearn
# Scale: Log salary scale

normalizer = sklearn.preprocessing.Normalizer(norm='l1')
X_train_norm = normalizer.fit_transform(X_train_counts.astype('float64'))
X_test_norm = normalizer.transform(X_test_counts.astype('float64'))
print(X_train_norm.shape)

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(
    alpha=.0001, penalty='l1', n_iter=10000)
sgd.fit(X_train_norm, Y_train)
sgd_predictions = sgd.predict(X_test_norm)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))

(120037, 10332)
SGDRegressor Mean Absolute Error: 10040.1751


In [28]:
# Model: SGD
# Data Representation: BOW
# Library used: sklearn
# Scale: Regular scale

X_train_index, X_test_index, Y_train, Y_test = (
    sklearn.model_selection.train_test_split(
        data.index, data['SalaryNormalized'],
        test_size=.3, random_state=42))

normalizer = sklearn.preprocessing.Normalizer(norm='l1')
X_train_norm = normalizer.fit_transform(X_train_counts.astype('float64'))
X_test_norm = normalizer.transform(X_test_counts.astype('float64'))
print(X_train_norm.shape)

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(
    alpha=.0001, penalty='l1', n_iter=1000)
sgd.fit(X_train_norm, Y_train)
sgd_predictions = sgd.predict(X_test_norm)
sgd_mae = mae_without_log_scaling(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))

(119970, 10027)
SGDRegressor Mean Absolute Error:  9408.0917


In [20]:
# Model: Random Forests using batch learning
# Data Representation: LDA
# Library used: sklearn

def batch_processed_rf(X_train, Y_train, batch_size=1000, n_estimators_per_batch=100):
    """Function to process random forest in batches using warm start."""
    
    def run_batch(X_train_batch, Y_train_batch, n_estimators, total_processed_rows):     
        """Function to run next batch of random forest."""
        processed_rows = total_processed_rows + len(X_train_batch)
        rf.set_params(n_estimators=n_estimators)
        rf.fit(X_train_batch, Y_train_batch)
        rf_predictions = rf.predict(X_test_lda)
        rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)
        print('Number of rows trained on: {:10.4f}'.format(processed_rows))
        print('Number of trees in forest: {:10.4f}'.format(n_estimators))
        print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))
        print('')
    
    n_estimators = n_estimators_per_batch
    total_processed_rows = 0
    
    # Define model.
    rf = sklearn.ensemble.RandomForestRegressor(
    n_estimators=n_estimators_per_batch, n_jobs=-1, warm_start=True)
    
    # Reindex training and test data.
    # Should randomize here, but we have already taken this step, so no need in our case.
    X_train.reset_index(drop=True)
    Y_train.reset_index(drop=True)
    
    while total_processed_rows < len(X_train) - batch_size:     
        # Scale to get Mean Absolute Error
        batch_index = range(total_processed_rows, total_processed_rows + batch_size)
        X_train_batch = X_train.iloc[batch_index]
        Y_train_batch = Y_train.iloc[batch_index]
        run_batch(X_train_batch, Y_train_batch, n_estimators, total_processed_rows)
        total_processed_rows += batch_size
        n_estimators += n_estimators_per_batch
        
    remaining_rows = len(X_train) - total_processed_rows
    if remaining_rows == 0:
        return
    batch_index = range(total_processed_rows, len(X_train))
    X_train_batch = X_train.iloc[batch_index]
    Y_train_batch = Y_train.iloc[batch_index]
    run_batch(X_train_batch, Y_train_batch, n_estimators, total_processed_rows)

batch_processed_rf(X_train_lda, Y_train, batch_size=1000, n_estimators_per_batch=100)

ValueError: Found input variables with inconsistent numbers of samples: [51416, 6000]

### Note: With the same data, training in batch results in a higher error

In [6]:
# Model: Random Forests using batch learning
# Data Representation: Bag of Words
# Library used: sklearn

# NOTE: This is just to get this set up and working.
# TODO: Move this to be a single function under 'Random Forests' where you simply define the
# model as being BOW or LDA.

def batch_processed_rf_bow(X_train, Y_train, batch_size=1000, n_estimators_per_batch=100):
    """Function to process random forest in batches using warm start."""
    # Define model.
    n_estimators = n_estimators_per_batch
    total_processed_rows = 0
    
    rf = sklearn.ensemble.RandomForestRegressor(
        n_estimators=n_estimators_per_batch, n_jobs=-1, warm_start=True)
   
    # Reindex training and test data.
    # Should randomize here, but we have already taken this step, so no need in our case.
    Y_train.reset_index(drop=True)
    
    def run_batch(X_train_batch, Y_train_batch, n_estimators, total_processed_rows):     
        """Function to run next batch of random forest."""
        # Transform the batch.
        processed_rows = total_processed_rows + len(X_train_batch)
        batch_X_train_count = count_vect.transform(X_train_batch[TRAINING_COLUMN])
        rf.set_params(n_estimators=n_estimators)
        rf.fit(batch_X_train_count, Y_train_batch)
        rf_predictions = rf.predict(X_test_counts)
        rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)
        print('Number of rows trained on: {:10.4f}'.format(processed_rows))
        print('Number of trees in forest: {:10.4f}'.format(n_estimators))
        print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))
        print('')
    
    while total_processed_rows < X_train.shape[0] - batch_size:     
        # Scale to get Mean Absolute Error
        batch_index = range(total_processed_rows, total_processed_rows + batch_size)
        X_train_batch = X_train.iloc[batch_index]
        Y_train_batch = Y_train.iloc[batch_index]
        run_batch(X_train_batch, Y_train_batch, n_estimators, total_processed_rows)
        total_processed_rows += batch_size
        n_estimators += n_estimators_per_batch
        
    remaining_rows = X_train.shape[0] - total_processed_rows
    if remaining_rows == 0:
        return
    batch_index = range(total_processed_rows, len(X_train))
    X_train_batch = X_train.iloc[batch_index]
    Y_train_batch = Y_train.iloc[batch_index]
    run_batch(X_train_batch, Y_train_batch, n_estimators, total_processed_rows)

batch_processed_rf_bow(X_train_bow, Y_train, batch_size=10000, n_estimators_per_batch=100)

Number of rows trained on: 10000.0000
Number of trees in forest:   100.0000
Random Forest Regressor Mean Absolute Error:  8663.3455

Number of rows trained on: 20000.0000
Number of trees in forest:   200.0000
Random Forest Regressor Mean Absolute Error:  8488.6272

Number of rows trained on: 30000.0000
Number of trees in forest:   300.0000
Random Forest Regressor Mean Absolute Error:  8437.6218

Number of rows trained on: 40000.0000
Number of trees in forest:   400.0000
Random Forest Regressor Mean Absolute Error:  8432.2712

Number of rows trained on: 50000.0000
Number of trees in forest:   500.0000
Random Forest Regressor Mean Absolute Error:  8406.4605

Number of rows trained on: 60000.0000
Number of trees in forest:   600.0000
Random Forest Regressor Mean Absolute Error:  8395.6568

Number of rows trained on: 70000.0000
Number of trees in forest:   700.0000
Random Forest Regressor Mean Absolute Error:  8396.6413

Number of rows trained on: 80000.0000
Number of trees in forest:   80